In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import convert_all as crd

In [2]:
# get all low_dim data
task_name = "example_cole"
raw_dir = f"../data/raw/{task_name}"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode=None,
    concatenater={
        "/observations/qpos": (
            "observation/arm/joint_position",
            "observation/eef/joint_position",
        ),
        "/action": (
            "action/arm/joint_position",
            "action/eef/joint_position",
        ),
    },
    key_filter=[
        "observation/eef/pose",
        "action/eef/pose",
        "/time",
    ],
)

Data Converting: 100%|██████████| 3/3 [00:00<00:00, 1253.03it/s]


In [ ]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")
print(list(data["0"].keys()))
print(list(data["0"]["/observations/qpos"][0]))
print(list(data["0"]["/action"][0]))

In [3]:
import os
import cv2

# merge high_dim data and save
raw_dir
# raw_names = ["observation.images.cam1", "observation.images.cam2"]
raw_names = ["observation.images.cam1"]
video_names = [name + ".mp4" for name in raw_names]
target_dir = f"../data/hdf5/{task_name}/"
low_dim_data = data
# name_converter = {"head": "/observations/images/0"}
name_converter = {raw_names[i]: f"/observations/images/{i}" for i in range(len(raw_names))}
print(f"name_converter: {name_converter}")
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))
max_pad_lenth = max(episode_lens)

episode_names = list(low_dim_data.keys())
print(f"Max episode length: {max_pad_lenth}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")

name_converter: {'observation.images.cam1': '/observations/images/0'}
Max episode length: 63
All episodes: ['1', '0', '2']
episode number: 3


In [4]:
def save_one(index, ep_name):
    crd.merge_video_and_save(
        low_dim_data[ep_name],
        f"{raw_dir}/{ep_name}",
        video_names,
        crd.save_dict_to_hdf5,
        name_converter,
        compresser,
        f"{target_dir}/" + target_namer(index),
        max_pad_lenth,
    )
    data.pop(ep_name)

In [6]:
from concurrent.futures import ThreadPoolExecutor

# save all data
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:

    def save_one(index, ep_name):
        crd.merge_video_and_save(
            low_dim_data[ep_name],
            f"{raw_dir}/{ep_name}",
            video_names,
            crd.save_dict_to_hdf5,
            name_converter,
            compresser,
            f"{target_dir}/" + target_namer(index),
            max_pad_lenth,
        )
        data.pop(ep_name)

    for index, ep_name in enumerate(episode_names):
        futures.append(executor.submit(save_one, index, ep_name))

In [8]:
save_one(0, episode_names[0])

KeyError: '1'